
<a href="https://colab.research.google.com/github/KAIET98/TFM_AGROTECH/blob/TBA_GOOGLE_FUNC/analytics/AEMET/GCP_SCRIPT/TBA_TRAINING/TBA_HISTORICO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carga de librerias

In [1]:
!pip install pymysql

In [2]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import pandas as pd
import numpy as np
import pymysql
#import cryptography
import pandas as pd
import logging

# Conexion a la base de datos

In [3]:

host = 'www.tbagrosensor.com'
db = 'tecfrut_ccl'
user = 'tecfrut_eae'
pasw = '.jvcUtJ?;='


'''
Nos conectamos a la base de datos
'''
logger = logging.getLogger()
logger.setLevel(logging.INFO)

try:
    conn = pymysql.connect(host=host, user=user, passwd=pasw, db=db, connect_timeout=5)
except pymysql.MySQLError as e:
    logger.error("ERROR: Unexpected error: Could not connect to MySQL instance.")
    logger.error(e)
    #sys.exit()

logger.info("SUCCESS: Connection to RDS MySQL instance succeeded")


# Funciones de bases de datos

In [4]:

def insert_into(data, arrival_df):

    #establecemos la conexión a la base de datos
    connection = pymysql.connect(host=host,
                             user=user,
                             password=pasw,
                             db=db)

    cursor = connection.cursor()


    #especificamos cual es el nombre de la tabla destino

    nombre_tabla = arrival_df

    
    # creating column list for insertion
    cols = "`,`".join([str(i) for i in data.columns.tolist()])
    cols = cols.strip()
    # Insert DataFrame recrds one by one.
    for i,row in data.iterrows():
        sql = "INSERT INTO `{}` (`".format(nombre_tabla) +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"

            
        cursor.execute(sql, tuple(row))

            # the connection is not autocommitted by default, so we must commit to save our changes
        connection.commit()


In [5]:

def run_query(q):
    with pymysql.connect(host=host,
                             user=user,
                             password=pasw,
                             db=db) as conn:
        return pd.read_sql(q, conn)

def checkTableExists(conn, tablename):
    dbcur = conn.cursor()
    dbcur.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(tablename.replace('\'', '\'\'')))
    if dbcur.fetchone()[0] == 1:
        dbcur.close()
        return True

    dbcur.close()
    return False


-

1.   Tenéis las 6 humedades de suelo en var1-var6. 
2.   Temperatura en var8, 
3.  humedad relativa en var9.
 Todos estos valores hay que dividirlos por 100.
4. Var12 es la velocidad de viento, es un contador por lo que lo importante es la diferencia da cada valor con la anterior lectura.
5. var13 es igual pero para la lluvia.
6. var14 es la dirección de viento.

El esquema objetivo sería el siguiente:


```

"""

CREATE TABLE {}(

  id_nodo VARCHAR(255), 
  HUMEDAD_1 integer, 
  HUMEDAD_2 integer, 
  HUMEDAD_3 integer, 
  HUMEDAD_4 integer, 
  HUMEDAD_5 integer, 
  HUMEDAD_6 integer, 
  COLUMNA_7 integer, 
  TEMPERATURA FLOAT, 
  HUMEDAD_RELATIVA FLOAT, 
  VELOCIDAD_VIENTO FLOAT, 
  LLUVIA FLOAT, 
  DIRECCION_VIENTO INTEGER,
  HORA TIME,
  FECHA TIMESTAMP(8)

)

""".format(tabla_no_existente)

```

In [6]:

'''
le tenemos que pasar :

1 = "-"
2 = "_"

'''

def tba_dia_formato(tipo):


  if tipo == 1:


    import re

    stmt = "SHOW TABLES"


    tablas = run_query(stmt)

    #renombramos la tabla de tablas
    tablas = tablas.rename(columns = {"Tables_in_tecfrut_ccl" : "TABLAS"})

    #filtramos


    regex = '^hist'

    tablas = tablas[tablas.TABLAS.str.match(regex)]



    tablas['fechas'] = tablas.TABLAS.str.extract('(\d+\_\d+\_\d+)')

    fechas_formato = []

    for fecha in range(tablas.shape[0]):

      fecha_a_modificar = tablas.iloc[fecha,1]

      fechas_formato.append(re.sub('_', '-', fecha_a_modificar))


    tablas['fechas_off'] = fechas_formato


    #nos quedamos con las columnas que nos interesan

    tablas = tablas[['fechas_off']]

    #los renombramos
    tablas = tablas.rename(columns = {'fechas_off':'fechas'})


    #transformamos a lista para obtener una funcionalidad mayor

    tablas = list(tablas['fechas'])

    return tablas

  elif tipo == 2: 

    import re

    stmt = "SHOW TABLES"


    tablas = run_query(stmt)

    #renombramos la tabla de tablas
    tablas = tablas.rename(columns = {"Tables_in_tecfrut_ccl" : "TABLAS"})

    #filtramos


    regex = '^hist'

    tablas = tablas[tablas.TABLAS.str.match(regex)]



    tablas['fechas'] = tablas.TABLAS.str.extract('(\d+\_\d+\_\d+)')


    #transformamos a lista para obtener una funcionalidad mayor

    tablas = list(tablas['fechas'])

    return tablas

  else: 

    print('''
    
    Por favor, indica con un 1 si quieres modificar el string de fecha de "_" a "-"
    o con un 2, si lo quires dejar en "_"
    
    ''')





#vemos el resultado 
tablas = tba_dia_formato(tipo = 1)


print(tablas[:10])

['2021-12-23', '2021-12-27', '2022-01-03', '2022-01-13', '2022-01-18', '2022-01-19', '2022-01-20', '2022-01-21', '2022-01-22', '2022-01-23']


# Creacion de la tabla de TBA_AGROSENSORS_HISTORICO

Primero de todo vamos a crear la tabla vacia, es decir el esquema

In [7]:



#NOS CONECTTAMOS  ANUESTRA BASE DE DATOS PARA CREAR EL HISTORICO 
host = '35.241.159.127' #este el el host nuevo
user = 'admin'
password = '12345678'
database = 'GET_DATABASE'


'''
Nos conectamos a la base de datos
'''
logger = logging.getLogger()
logger.setLevel(logging.INFO)

try:
    conn = pymysql.connect(host=host, user=user, passwd=pasw, db=db, connect_timeout=5)
except pymysql.MySQLError as e:
    logger.error("ERROR: Unexpected error: Could not connect to MySQL instance.")
    logger.error(e)
    #sys.exit()

logger.info("SUCCESS: Connection to RDS MySQL instance succeeded")

tabla_no_existente = "TBA_AGROSENSORS_HISTORIC_TRAINING"

#Si es la primera vez que lanzamos la consuilta, creamos a tabla, 
# si la tabla esta creada, simplemente vemos que infomracion tiene
if checkTableExists(conn, tabla_no_existente) == False:


 

  q = """CREATE TABLE {}(

  id_nodo VARCHAR(255), 
  HUMEDAD_1 integer, 
  HUMEDAD_2 integer, 
  HUMEDAD_3 integer, 
  HUMEDAD_4 integer, 
  HUMEDAD_5 integer, 
  HUMEDAD_6 integer, 
  COLUMNA_7 integer, 
  TEMPERATURA FLOAT, 
  HUMEDAD_RELATIVA FLOAT, 
  VELOCIDAD_VIENTO FLOAT, 
  LLUVIA FLOAT, 
  DIRECCION_VIENTO INTEGER,
  HORA TIME,
  FECHA TIMESTAMP(6)

  )

  """.format(tabla_no_existente)

  print(run_query(q))






#vemos el resultado

q = """

select * from {};

""".format(tabla_no_existente)


tba_historico = run_query(q)

if tba_historico.shape[0] == 0:

  print('La tabla está vacia')


else:

  print(tba_historico.head())

  id_nodo  HUMEDAD_1  HUMEDAD_2  HUMEDAD_3  HUMEDAD_4  HUMEDAD_5  HUMEDAD_6  \
0       1          0          0          0          0          0          0   
1       1          0          0          0          0          0          0   
2       1          0          0          0          0          0          0   
3       1          0          0          0          0          0          0   
4       1          0          0          0          0          0          0   

   COLUMNA_7  TEMPERATURA  HUMEDAD_RELATIVA  VELOCIDAD_VIENTO  LLUVIA  \
0          0        16.68             69.65               0.0     0.0   
1          0        17.36             67.05               0.0     0.0   
2          0        17.33             66.37               0.0     0.0   
3          0        17.29             66.69               0.0     0.0   
4          0        17.26             66.89               0.0     0.0   

   DIRECCION_VIENTO            HORA      FECHA  
0                12 0 days 11:43:51 2

## Insertando datos a TBA_AGROSENSORS_HISTORICO

In [8]:
q = """

select fecha from {}

""".format('METEREOLOGIA')


metereologia_test_fechas = run_query(q)


metereologia_test_fechas

,fecha
0,1998-01-01
1,1998-01-02
2,1998-01-03
3,1998-01-04
4,1998-01-05
...,...
9313,2022-03-25
9314,2022-03-26
9315,2022-03-26
9316,2022-03-27


La información en TBA, incluye el dia, pero esta i ncrustado en la fecha, por lo que para comparar ambas bases de datos tendremos que crear una columna aparte
de la query que hagamos a TBA para sacar el dia. Lo que haremos también va a ser extraer la hora, para tener un registro horario también guardado.

In [9]:

host = 'www.tbagrosensor.com'
db = 'tecfrut_ccl'
user = 'tecfrut_eae'
pasw = '.jvcUtJ?;='




def informacion_dia(dia, host, db, user, pasw):

  host = host
  db = db
  user = user
  pasw = pasw


  '''
  Nos conectamos a la base de datos
  '''
  logger = logging.getLogger()
  logger.setLevel(logging.INFO)

  try:
      conn = pymysql.connect(host=host, user=user, passwd=pasw, db=db, connect_timeout=5)
  except pymysql.MySQLError as e:
      logger.error("ERROR: Unexpected error: Could not connect to MySQL instance.")
      logger.error(e)
      #sys.exit()

  logger.info("SUCCESS: Connection to RDS MySQL instance succeeded")


  #construimos el string del dia

  dia = 'historico_graficas_{}'.format(dia)

  q = """

  select * from {}

  """.format(dia)


  fecha_X = run_query(q)

  # 1. EXTRACCION FECHA --------------------------

  # Tranformamos la fecha a un feature más manejable. esd ecir
  # al que tenemos en la tabla METEREOLOGIA, datos test.


  instante = pd.DataFrame(fecha_X['instante'])

  fecha_X['instante'] = instante.applymap(lambda x: str(x))



  fecha_X['DIA'] = fecha_X.instante.str.extract('(\d+\-\d+\-\d+)')


  # 2. Extracción HORA-------------------------------------------


  
  hora = pd.DataFrame(fecha_X['instante'])

  fecha_X['hora'] = hora.applymap(lambda x: str(x))


  fecha_X['hora'] = fecha_X['hora'].str.extract('(\d+\:\d+\:\d+)')






  #seleccionamos las columnas


  fecha_X = fecha_X[['id_nodo', 	'var1', 	'var2', 	'var3', 	'var4', 	'var5', 	'var6', 	'var7',  'var8',	'var9', 'var12', 'var13', 'var14', 'hora', 'DIA']]


  # renommbramos las columnas

  '''
  id_nodo VARCHAR(255), 
      HUMEDAD_1 integer, 
      HUMEDAD_2 integer, 
      HUMEDAD_3 integer, 
      HUMEDAD_4 integer, 
      HUMEDAD_5 integer, 
      HUMEDAD_6 integer, 
      COLUMNA_7 integer, 
      TEMPERATURA FLOAT, 
      HUMEDAD_RELATIVA FLOAT, 
      VELOCIDAD_VIENTO FLOAT, 
      LLUVIA FLOAT, 
      DIRECCION_VIENTO INTEGER,
      FECHA TIMESTAMP(6), 

      DIA DATE
  '''



  fecha_X = fecha_X.rename(columns = {
      
      'var1' : 'HUMEDAD_1', \
      'var2' : 'HUMEDAD_2', \
      'var3' : 'HUMEDAD_3', \
      'var4' : 'HUMEDAD_4', \
      'var5' : 'HUMEDAD_5', \
      'var6' : 'HUMEDAD_6', \
      'var7' : 'COLUMNA_7', \
      'var8' : 'TEMPERATURA', \
      'var9' : 'HUMEDAD_RELATIVA', \
      'var12' : 'VELOCIDAD_VIENTO', \
      'var13' : 'LLUVIA', \
      'var14' : 'DIRECCION_VIENTO', \
     # 'var15' : 'FECHA', \
      'DIA':'FECHA', \
      'hora': 'HORA'
      
                                      })
  

  #3. Adecuación de datos -----------------------------
  '''
  En el correo puso que todos los valores habia que dividirlos por 100 por lo que
  tranformamos los datos

  '''

  fecha_X['HUMEDAD_1'] = fecha_X['HUMEDAD_1'] / 100
  fecha_X['HUMEDAD_2'] = fecha_X['HUMEDAD_2'] / 100
  fecha_X['HUMEDAD_3'] = fecha_X['HUMEDAD_3'] / 100
  fecha_X['HUMEDAD_4'] = fecha_X['HUMEDAD_4'] / 100
  fecha_X['HUMEDAD_5'] = fecha_X['HUMEDAD_5'] / 100
  fecha_X['HUMEDAD_6'] = fecha_X['HUMEDAD_6'] / 100
  fecha_X['COLUMNA_7'] = fecha_X['COLUMNA_7'] / 100
  fecha_X['TEMPERATURA'] = fecha_X['TEMPERATURA'] / 100
  fecha_X['HUMEDAD_RELATIVA'] = fecha_X['HUMEDAD_RELATIVA'] / 100
  fecha_X['VELOCIDAD_VIENTO'] = fecha_X['VELOCIDAD_VIENTO'] / 100
  fecha_X['LLUVIA'] = fecha_X['LLUVIA'] / 100
  fecha_X['DIRECCION_VIENTO'] = fecha_X['DIRECCION_VIENTO'] / 100



  return fecha_X





In [10]:
tablas = tba_dia_formato(tipo  = 2)


#nos quedamos con el primer elemento 

fecha = tablas[0]

Falta hacer la prueba de concepto

In [11]:

historico_limpio = informacion_dia(fecha, host, db, user, pasw)

historico_limpio.head()

,id_nodo,HUMEDAD_1,HUMEDAD_2,HUMEDAD_3,HUMEDAD_4,HUMEDAD_5,HUMEDAD_6,COLUMNA_7,TEMPERATURA,HUMEDAD_RELATIVA,VELOCIDAD_VIENTO,LLUVIA,DIRECCION_VIENTO,HORA,FECHA
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.68,69.65,0.0,0.0,11.94,11:43:51,2021-12-23
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.36,67.05,0.0,0.0,14.65,11:53:10,2021-12-23
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.33,66.37,0.0,0.0,12.85,12:02:36,2021-12-23
3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.29,66.69,0.0,0.0,12.12,12:13:31,2021-12-23
4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.26,66.89,0.0,0.0,12.09,12:22:51,2021-12-23


# Subimos el historico de TBA

In [12]:
host = '34.79.75.171'
db = 'GET_DATABASE'
user = 'admin'
pasw = '12345678'


'''
Nos conectamos a la base de datos
'''
logger = logging.getLogger()
logger.setLevel(logging.INFO)

try:
    conn = pymysql.connect(host=host, user=user, passwd=pasw, db=db, connect_timeout=5)
except pymysql.MySQLError as e:
    logger.error("ERROR: Unexpected error: Could not connect to MySQL instance.")
    logger.error(e)
    #sys.exit()

logger.info("SUCCESS: Connection to RDS MySQL instance succeeded")


q = """

select * from {}

""".format(tabla_no_existente)


metereologia_test_fechas = run_query(q)


if metereologia_test_fechas.shape[0] == 0:

    print('Subimos el historico')

    host = 'www.tbagrosensor.com'
    db = 'tecfrut_ccl'
    user = 'tecfrut_eae'
    pasw = '.jvcUtJ?;='




    '''
    Nos conectamos a la base de datos
    '''
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    try:
        conn = pymysql.connect(host=host, user=user, passwd=pasw, db=db, connect_timeout=5)
    except pymysql.MySQLError as e:
        logger.error("ERROR: Unexpected error: Could not connect to MySQL instance.")
        logger.error(e)
        #sys.exit()

    logger.info("SUCCESS: Connection to RDS MySQL instance succeeded")


    #Nos quedamos con los nombres de las tablas, pero la parte d elas fechas


    tablas = tba_dia_formato(tipo = 2)


    #por cada fecha

    for tabla in tablas:

      #construimos el nombre de la tabla a filtrar

      print('\n Subiremos la tabla: ')

      print('historico_graficas_{}'.format(tabla), '\n')


      #nos quedamos con el nombre de la tabla

      tabla_nomobre = 'historico_graficas_{}'.format(tabla)


      #llamamos a la base de datos de TBA

      host = 'www.tbagrosensor.com'
      db = 'tecfrut_ccl'
      user = 'tecfrut_eae'
      pasw = '.jvcUtJ?;='

      historico_limpio = informacion_dia(tabla, host, db, user, pasw)

      print(historico_limpio.head(2))




      

      print('\n', 'Subimos el historico de la tabla {}'.format(tabla_nomobre), '\n')

      '''

      Antes de subir nada, nos conectamos a nuesta base se datos de GET

      '''
      host = '34.79.75.171'
      db = 'GET_DATABASE'
      user = 'admin'
      pasw = '12345678'


      '''
      Nos conectamos a la base de datos
      '''
      logger = logging.getLogger()
      logger.setLevel(logging.INFO)

      try:
          conn = pymysql.connect(host=host, user=user, passwd=pasw, db=db, connect_timeout=5)
      except pymysql.MySQLError as e:
          logger.error("ERROR: Unexpected error: Could not connect to MySQL instance.")
          logger.error(e)
          #sys.exit()

      logger.info("SUCCESS: Connection to RDS MySQL instance succeeded", "\n")






      insert_into(historico_limpio, tabla_no_existente)




      print('##############')

